In [ ]:
import pandas as pd
df = pd.read_csv("report.csv")

In [4]:
sdf = df.sort_values('rt', ascending=False)

In [9]:
pd.set_option('display.max_colwidth', -1)
hdf = sdf[['date','request','rt']].head(100)

In [11]:
import urllib
hdf['request'] = hdf['request'].map(urllib.parse.unquote)
hdf

,date,request,rt
1803647,04/Sep/2020:11:05:01 +0000,"GET /fr/search/travelq?f[0]=:Commissariats à l’information et à la protection de la vie privée au Canada&f[1]=travelq_total_fr:Adjointe spéciale, opérations et planification&f[2]=travelq_total_fr:CSPPPM&f[3]=travelq_total_fr:Directeur exécutif et avocat général, Secrétariat du Tribunal canadien du commerce extérieur&f[4]=travelq_total_fr:Sous-ministre adjoint Direction générale de la gestion intégrée&f[5]=travelq_total_fr:Sous-ministre adjointe intérimaire, Secteur de la gestion des affaires publiques et du portefeuille&f[6]=travelq_total_fr:Vice-président, Affaires extérieures et développement d'entreprise HTTP/1.1",76.953
2771972,05/Sep/2020:15:33:21 +0000,"GET /fr/search/travelq?f[0]=:Conseil de recherches en sciences naturelles et en génie du Canada&f[1]=travelq_total_fr:Assistante à la Conseillère principale, Gestion des enjeux&f[2]=travelq_total_fr:Ministre du Développement économique et des Langues officielles&f[3]=travelq_total_fr:Sous-ministre adjoint, Politique stratégique, planification et affaires ministérielles&f[4]=travelq_total_fr:Sous-ministre adjoint principal, Services ministériels et dirigeant principal des finances &f[5]=travelq_total_fr:VP, Services corporatifs et Chef de la direction financière&f[6]=travelq_total_fr:Vice-présidente, Direction générale des affaires autochtones et du patrimoine culturel&f[7]=travelq_total_fr:Vice-présidente par intérim, Nouveau-Brunswick HTTP/1.1",75.058
2158831,07/Sep/2020:14:40:56 +0000,"GET /fr/search/travelq?f[0]=:Tribunal des anciens combattants (révision et appel)&f[1]=travelq_total_fr:Adjoint spécial, Région de l'Atlantique&f[2]=travelq_total_fr:Secrétaire parlemantaire&f[3]=travelq_total_fr:Sous-commissaire Principal, Administration Centrale&f[4]=travelq_total_fr:Vice-présidente, conseillère spéciale&f[5]=travelq_total_fr:Vice-présidente adjointe, SPIIE HTTP/1.1",71.040
1789164,04/Sep/2020:07:33:18 +0000,"GET /fr/search/travelq?f[0]=:Conseil de recherches en sciences naturelles et en génie du Canada&f[1]=travelq_total_fr:Ministre du Développement économique et des Langues officielles&f[2]=travelq_total_fr:Présidente-didrectrice générale&f[3]=travelq_total_fr:Secrétaire parlementaire pour l'Infrastructure et les Collectivités&f[4]=travelq_total_fr:Sous-ministre adjoint délégué, Direction des finances et échanges internationaux&f[5]=travelq_total_fr:Sous-ministre ajointe, Direction générale des programmes &f[6]=travelq_total_fr:VP, Services corporatifs et Chef de la direction financière&f[7]=travelq_total_fr:Vice-présidente par intérim, Nouveau-Brunswick HTTP/1.1",70.623
709777,09/Sep/2020:10:30:50 +0000,"GET /fr/search/travelq?f[0]=:Tribunal des anciens combattants (révision et appel)&f[1]=travelq_total_fr:Administrateur en chef des opérations&f[2]=travelq_total_fr:Ministre de l’Innovation, des Sciences et de l'Industrie&f[3]=travelq_total_fr:SPSC pour le Personnel Executif&f[4]=travelq_total_fr:Vice-présidente, Radiodiffusion &f[5]=travelq_total_fr:Vice président et Dirigeant principal HTTP/1.1",69.960
3094264,08/Sep/2020:15:50:10 +0000,"GET /fr/search/travelq?f[0]=:Commission civile d'examen et de traitement des plaintes relatives à la Gendarmerie royale du Canada&f[1]=travelq_total_fr:Chef - Réserves et appui de l'employeur (CRAE)&f[2]=travelq_total_fr:Conseiller politique principal&f[3]=travelq_total_fr:Directeur des politiques et affaires parlementaires&f[4]=travelq_total_fr:Sous-ministre adjoint, Direction générales des services à l'industrie et aux marchés&f[5]=travelq_total_fr:Vice président HTTP/1.1",69.785
709778,09/Sep/2020:10:30:50 +0000,"GET /fr/search/travelq?f[0]=:École de la fonction publique du Canada&f[1]=travelq_total_fr:Adjointe spéciale - Affaires parlementaires&f[2]=travelq_total_fr:Sous-ministre adjointe, Secteur des terres et des minéraux HTTP/1.1",69.156
1284643,07/Sep/2020:06:24:48 +0000,"GET /fr/search/travelq?f[0]=:Commissariats à l’information et à la protection de la vie privée au Cana